In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
color = sns.color_palette()

import subprocess
from subprocess import check_output
import gc 
%matplotlib inline 

In [2]:
files = check_output(["ls", "Data/"]).decode("utf8")
print(files)

holidays_events.csv
items.csv
oil.csv
sample_submission.csv
stores.csv
test.csv
train.csv
transactions.csv



In [6]:
for file in files.split("\n"):
    path = "Data/" + file
    try:
        with open(path, 'r') as f:
            lines = f.readlines()
        print("The file: ", file, "has: ", len(lines), "rows")
    except:
        print("The file: ", file, "was not found.")

The file:  holidays_events.csv has:  351 rows
The file:  items.csv has:  4101 rows
The file:  oil.csv has:  1219 rows
The file:  sample_submission.csv has:  3370465 rows
The file:  stores.csv has:  55 rows
The file:  test.csv has:  3370465 rows
The file:  train.csv has:  125497041 rows
The file:  transactions.csv has:  83489 rows
The file:   was not found.


In [7]:
Dir = "Data"
test = pd.read_csv(f"{Dir}/test.csv")
stores = pd.read_csv(f"{Dir}/stores.csv")
items = pd.read_csv(f"{Dir}/items.csv")
trans = pd.read_csv(f"{Dir}/transactions.csv")
oil = pd.read_csv(f"{Dir}/oil.csv")
holiday = pd.read_csv(f"{Dir}/holidays_events.csv")
print("done")

done


In [8]:
mem_test = test.memory_usage(index=True).sum()
print("Dataset uses: ",mem_test/1024**2," MB")
print(test.info(memory_usage="deep"))

Dataset uses:  106.07284545898438  MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370464 entries, 0 to 3370463
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   id           int64 
 1   date         object
 2   store_nbr    int64 
 3   item_nbr     int64 
 4   onpromotion  bool  
dtypes: bool(1), int64(3), object(1)
memory usage: 295.7 MB
None


In [9]:
test['store_nbr'] = test['store_nbr'].astype(np.uint8)
test['id'] = test['id'].astype(np.uint32)
test['item_nbr'] = test['item_nbr'].astype(np.uint32)
test['date'] = pd.to_datetime(test['date'], format="%Y-%m-%d")
print(test.memory_usage(index=True))
new_mem_test = test.memory_usage(index=True).sum()
print("test dataset uses ", new_mem_test / 1024**2, " MB after changes")
print("memory saved =",(mem_test-new_mem_test) / 1024**2," MB")

Index               128
id             13481856
date           26963712
store_nbr       3370464
item_nbr       13481856
onpromotion     3370464
dtype: int64
test dataset uses  57.85797119140625  MB after changes
memory saved = 48.214874267578125  MB


In [11]:
dtype_dict = {"id": np.uint32,
              "store_nbr": np.uint8,
              "item_nbr": np.uint32,
              "unit_sales": np.float16
              }

train_part1 = pd.read_csv(f"{Dir}/train.csv", nrows=100,
                          dtype=dtype_dict, usecols=[0, 2, 3, 4])
print(train_part1.describe())
print(train_part1.dtypes)


train_part2 = pd.read_csv(f"{Dir}/train.csv", nrows=10000,
                          dtype=dtype_dict, usecols=[1, 5], parse_dates=[0])
train_part2['Year'] = pd.DatetimeIndex(train_part2['date']).year
train_part2['Month'] = pd.DatetimeIndex(train_part2['date']).month
train_part2['Day'] = pd.DatetimeIndex(train_part2['date']).day.astype(np.uint8)
del (train_part2['date'])
train_part2['Day'] = train_part2['Day'].astype(np.uint8)
train_part2['Month'] = train_part2['Month'].astype(np.uint8)
train_part2['Year'] = train_part2['Year'].astype(np.uint16)

train_part2["onpromotion"].fillna(0, inplace=True)
train_part2["onpromotion"] = train_part2["onpromotion"].astype(np.int8)
print(train_part2.head())
print(train_part2.dtypes)

               id  store_nbr       item_nbr  unit_sales
count  100.000000      100.0     100.000000  100.000000
mean    49.500000       25.0  186736.120000    5.019531
std     29.011492        0.0   56520.574966   10.515625
min      0.000000       25.0  103665.000000    1.000000
25%     24.750000       25.0  128208.000000    1.000000
50%     49.500000       25.0  172263.500000    2.000000
75%     74.250000       25.0  231846.250000    5.000000
max     99.000000       25.0  275823.000000   90.000000
id             uint32
store_nbr       uint8
item_nbr       uint32
unit_sales    float16
dtype: object
   onpromotion  Year  Month  Day
0            0  2013      1    1
1            0  2013      1    1
2            0  2013      1    1
3            0  2013      1    1
4            0  2013      1    1
onpromotion      int8
Year           uint16
Month           uint8
Day             uint8
dtype: object


In [12]:
train_part2 = pd.read_csv(f"{Dir}/train.csv",
                          dtype=dtype_dict, usecols=[1, 5], parse_dates=[0])
train_part2['Year'] = pd.DatetimeIndex(train_part2['date']).year
train_part2['Month'] = pd.DatetimeIndex(train_part2['date']).month
train_part2['Day'] = pd.DatetimeIndex(train_part2['date']).day.astype(np.uint8)
del (train_part2['date'])
train_part2['Day'] = train_part2['Day'].astype(np.uint8)
train_part2['Month'] = train_part2['Month'].astype(np.uint8)
train_part2['Year'] = train_part2['Year'].astype(np.uint16)

train_part2["onpromotion"].fillna(0, inplace=True)
train_part2["onpromotion"] = train_part2["onpromotion"].astype(np.int8)
print(train_part2.head())
print(train_part2.dtypes)

C:\Users\anisr\AppData\Local\Temp\ipykernel_21316\4016694863.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_part2 = pd.read_csv(f"{Dir}/train.csv",


   onpromotion  Year  Month  Day
0            0  2013      1    1
1            0  2013      1    1
2            0  2013      1    1
3            0  2013      1    1
4            0  2013      1    1
onpromotion      int8
Year           uint16
Month           uint8
Day             uint8
dtype: object


In [13]:
dtype_dict = {"id": np.uint32,
              "store_nbr": np.uint8,
              "item_nbr": np.uint32,
              "unit_sales": np.float32
              }

train_part1 = pd.read_csv(f"{Dir}/train.csv",
                          dtype=dtype_dict, usecols=[0, 2, 3, 4])
print(train_part1.dtypes)

id             uint32
store_nbr       uint8
item_nbr       uint32
unit_sales    float32
dtype: object


In [14]:
train = pd.concat([train_part1.reset_index(drop=True), train_part2], axis=1)
del (train_part1)
del (train_part2)
id = train['id']
del (train['id'])
print(train.memory_usage())

mem_train = 5*1024**3
new_mem_train = train.memory_usage().sum()
print("Train dataset uses ", new_mem_train / 1024**2, " MB after changes")
print("memory saved is approx", (mem_train-new_mem_train) / 1024**2, " MB")

Index                128
store_nbr      125497040
item_nbr       501988160
unit_sales     501988160
onpromotion    125497040
Year           250994080
Month          125497040
Day            125497040
dtype: int64
Train dataset uses  1675.5663757324219  MB after changes
memory saved is approx 3444.433624267578  MB
